In [1]:
from pathlib import Path
import subprocess
from concurrent.futures import ThreadPoolExecutor

def process_l2_image(file, output_dir):
    print(f"Processing L2 image: {file}")
    cmd = ['python','l2_processor/l2_master.py', str(file), '--cal', 'l2_config.json', '--outdir', str(output_dir)]
    result = subprocess.run(cmd, check=True, shell=True)
    return result


In [2]:
# ***************** Input images *******************
files = list(Path('Y:/06_SpaceStuff/Spacecraft/Proba3/data/Orbit273/P3SC_L1_20251217_222845/').rglob('aspiics_wb_l1*.fits'))
files.sort()

# for file in files:
#     process_l2_image(file, Path('Y:/06_SpaceStuff/Spacecraft/Proba3/processed/Orbit273'))

# Alternatively, to process images in parallel using threads:
threads = 12
with ThreadPoolExecutor(max_workers=threads) as executor:
    futures = [executor.submit(process_l2_image, file, Path('Y:/06_SpaceStuff/Spacecraft/Proba3/processed/Orbit273/')) for file in files]
    for future in futures:
        future.result()  # to catch exceptions if any



Processing L2 image: Y:\06_SpaceStuff\Spacecraft\Proba3\data\Orbit273\P3SC_L1_20251217_222845\aspiics_wb_l1_11108000000111_20250716T163506.fitsProcessing L2 image: Y:\06_SpaceStuff\Spacecraft\Proba3\data\Orbit273\P3SC_L1_20251217_222845\aspiics_wb_l1_11108000000112_20250716T163507.fits

Processing L2 image: Y:\06_SpaceStuff\Spacecraft\Proba3\data\Orbit273\P3SC_L1_20251217_222845\aspiics_wb_l1_11108000000113_20250716T163508.fits
Processing L2 image: Y:\06_SpaceStuff\Spacecraft\Proba3\data\Orbit273\P3SC_L1_20251217_222845\aspiics_wb_l1_11108000000211_20250716T163536.fits
Processing L2 image: Y:\06_SpaceStuff\Spacecraft\Proba3\data\Orbit273\P3SC_L1_20251217_222845\aspiics_wb_l1_11108000000212_20250716T163537.fits
Processing L2 image: Y:\06_SpaceStuff\Spacecraft\Proba3\data\Orbit273\P3SC_L1_20251217_222845\aspiics_wb_l1_11108000000213_20250716T163538.fits
Processing L2 image: Y:\06_SpaceStuff\Spacecraft\Proba3\data\Orbit273\P3SC_L1_20251217_222845\aspiics_wb_l1_11108000000311_20250716T1636